#### Reference:
[Get Started with Eager Execution](https://www.tensorflow.org/get_started/eager)

#### 目標：熟悉Tensorflow寫法,讀取iris(鳶尾花)資料集,建立模型來預測資料屬於哪一種iris花

主要流程：
1. 匯入並整理資料
2. 選擇適合的model
3. 訓練(train) model
4. 評估(evaluate) model效果
5. 利用訓練後的模型來預測

不像一般Tensorflow需要先建立計算圖才能執行,此處使用Eager execution來開發,可以立即得到執行結果
>一旦Eager execution設定開啟,則此程式的Eager execution就無法關閉(除非重啟)

In [1]:
# 此行適用於python2的環境, python3已經內建
# from __future__ import absolute_import, division, print_function

# python os操作及繪圖用
import os
import matplotlib.pyplot as plt

# import Tensorflow(tf) and Tensorflow Eager(tfe)
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# 要確認tf version >1.7
print('Tensorflow Version:{}'.format(tf.VERSION))

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Tensorflow Version:1.7.0


In [2]:
# 開啟Eager execution
tf.enable_eager_execution()
print('Eager Version:{}'.format(tf.executing_eagerly()))

Eager Version:True


#### 問題描述：
* Iris flower有三種分類:
    * Iris setosa
    * Iris virginica
    * Iris versicolor
* 我們依據底下特徵來訓練/訓練模型:
    * sepals(花萼長度)
    * petals(花瓣長度)

In [3]:
# 填入資料所在的網址
url="http://download.tensorflow.org/data/iris_training.csv"

# 用keras(包裝tensorflow,簡化使用方式的deep learning Library)的工具去抓取資料到本機
# 預設會下載在: /Users/[your_username]/.keras/datasets/iris_training.csv
# 此方法會將下載路徑傳回
# 若下載目錄已經有需要的資料,就不會再重新下載
download_path=tf.keras.utils.get_file(fname=os.path.basename(url),origin=url)
print('資料存放在本機的目錄為:{}'.format(download_path))

資料存放在本機的目錄為:/Users/seawnm/.keras/datasets/iris_training.csv


In [4]:
# 用unix/linux command來看一下前5筆資料集(用參數n來指定)
# 第一列資料,作者告訴我們一共120筆資料,4個特徵值,預測三種花(setosa,versicolor,virginica)
# 第二列資料開始,前4欄為特徵值及花的種類(0,1,2表示setosa,versicolor,virginica)
!head -n 5 {download_path}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


In [5]:
# 定義函式來處理csv資料
def parse_csv(line):
#   前四筆為浮點數,最後一個為整數
    layout=[[0.],[0.],[0.],[0.],[0]]
#   利用tf的DataSet API轉換csv資料
    parsed_line=tf.decode_csv(line,layout)
#   取出特徵值,[:-1]代表最後一個欄位不取,其他欄位都取出
    features=tf.reshape(parsed_line[:-1], shape=(4,))
#   取出標籤,[-1]代表取出最後一欄
    label=tf.reshape(parsed_line[-1],shape=())
    return features,label

In [6]:
# 設定資料讀取器(tf裡雖然習慣稱dataset,但實際上是一個能儲存並處理dataset的object)
# 讀入方式為每次一列
train_dataset=tf.data.TextLineDataset(download_path)
# 略過第一行說明不需匯入
train_dataset=train_dataset.skip(1)
# 將每一列的值丟入上面定義的函式處理成我們要的格式
train_dataset=train_dataset.map(parse_csv)
# 打亂資料列的順序(增加隨機性,避免影響模型訓練)
train_dataset=train_dataset.shuffle(buffer_size=1000)
# 將每32筆資料分為1個子集,訓練時每個子集算完就更新模型權重,可以增加訓練效率
# 一般更新權重的方式為: 梯度下降法(Gradient Descent)
# 分幾個子集分別更新則叫批次梯度下降法(Batch Gradient Descent)
train_dataset=train_dataset.batch(32)

# 執行讀取資料,用iterator物件的next函式依次傳回每一列資料物件
features, label=tfe.Iterator(train_dataset).next()

print('Example feature:',features[0])
print('Example label:',label[0])
# 若用format的方式印,會印不出tf的資訊
print('\nExample feature with format():{}'.format(features[0]))

Example feature: tf.Tensor([5.4 3.7 1.5 0.2], shape=(4,), dtype=float32)
Example label: tf.Tensor(0, shape=(), dtype=int32)

Example feature with format():[5.4 3.7 1.5 0.2]


In [7]:
# 利用Keras建構Multilayer Perceptron (MLP)類神經網路模型
# 一共建立三層,前兩層各有10個節點,最後一層節點數為3(即要分類的種類數)
# 第一層需要指定輸入的特徵數為4,後面每一層的輸入特徵數keras會自動推算
# 每個節點計算後的激活函式(activation)用Relu,限縮節點算出的值域最小為0
# activation可以視情況用不同方法,Relu為隱藏層中(除了最後一層),目前研究上效果不錯的方式之一
# 節點數及層數越多通常越精準,但也會耗費計算時間,也會有overfitting(太過擬和訓練數據)等問題
model=tf.keras.Sequential([
    tf.keras.layers.Dense(10,activation='relu',input_shape=(4,))
    ,tf.keras.layers.Dense(10,activation='relu')
    ,tf.keras.layers.Dense(3)
])

訓練網路前,需要定義損失函數(loss function)決定了model如何衡量預測跟實際label的誤差

Iris花的問題在此視為監督式的機器學習問題處理(即已知資料的label下,要去預測未知資料的label)

In [8]:
# 定義損失函數(量化預測跟實際之間的誤差)
# x:features data, y:label data
def loss(model, x, y):
#   y_儲存model預測出的label
    y_=model(x)
#   softmax方法用於表示多重分類問題(超過2種)的model輸出,計算每一種分類可能的機率後,將最高機率的種類標示為1,其他為0
#   cross entropy方法用於計算多種分類的誤差值
    return tf.losses.sparse_softmax_cross_entropy(label=y, logits=y_)

In [10]:
# 定義最佳化函數,作為每一次model更新權重時的依據,以減少loss值
# 一般常用批次梯度下降法
def grad(model, inputs, targets):
#   使用GradientTape紀錄損失函數
    with tfe.GradientTape() as tape:
        loss_value=loss(model, inputs, targets)
#   啟動梯度下降法透過更新model.variables, 以求得loss_value最小
#   tf中神經網路各節點的權重(Weights)及誤差項(Bias)都用tf.Variables型態表示
#   比如linear model= Bias + Weights x Features
#   Keras建構的model由tf構成,所以可以叫出variables
    return tape.gradient(loss_value, model.variables)

In [ ]:
# standard gradient descent (SGD)